# Assess the likelihood of a recurrent deletions in Alpha, BA.1 and BA.2
It is plausible that the SARS-CoV-2 deletions at positions 11288-11296 (BA.2 and Alpha) and 11283-11291 (BA.1) do not represent 3 separate origins. Here we re-run the sc2ts HMM to see if there are alternative likely explanations

In [1]:
import os

import numpy as np
import tskit
import tszip
import sc2ts

ts_dir = "../data"
ts_file = "find_problematic_v2-2023-01-01.ts.il.dated.tsz"
ds_file = "viridian_2024-04-29.alpha1.zarr.zip"
ds = sc2ts.Dataset(os.path.join(ts_dir, ds_file))
print("Loaded", ds)
ts = tszip.decompress(os.path.join(ts_dir, ts_file))
ts

Loaded Dataset at ../data/viridian_2024-04-29.alpha1.zarr.zip with 3960704 samples and 31 metadata fields


In [2]:
BA_1 = "ERR7602255"
BA_2 = "SRR17712694"

Alpha_node = 78090
BA_1_node = 316856
BA_2_node = 346163  # origin is recombination node 346185
assert np.sum(ts.mutations_node == Alpha_node) == 17
assert np.sum(ts.mutations_node == BA_1_node) >= 30
assert ts.node(BA_1_node).metadata["strain"] == BA_1
assert ts.node(BA_2_node).metadata["strain"] == BA_2
assert np.sum(ts.mutations_node == 346185) >= 30

In [3]:
ts_pre_BA_1 = tszip.decompress(os.path.join(ts_dir, "find_problematic_v2-2021-11-25.ts.tsz"))
ts_pre_BA_2 = tszip.decompress(os.path.join(ts_dir, "find_problematic_v2-2021-12-21.ts.tsz"))

In [4]:
nmut = 3  # how many SNP-like mutations do we place to represent a deletion
BA_1_haplotype = ds.haplotypes[BA_1]
BA_2_haplotype = ds.haplotypes[BA_2]  # Just pick an early one at random
# Check all alleles are the same across all sites
assert np.all(ds["variant_allele"][:] == ds["variant_allele"][0, :])

print(f"BA.1: {BA_1}")
# Move the deletion in BA.1 from position 11283 to 11288, and mark the rest as missing
print(
    "Was     ", 
    "".join(ds["variant_allele"][:][np.arange(len(BA_1_haplotype)), BA_1_haplotype][11280: 11300])
)
# Shift the first 5 letters
BA_1_haplotype[11282: 11287] = BA_1_haplotype[11291: 11296]
BA_1_haplotype[11291: 11296] = BA_1_haplotype[11287]
print(
    "Now     ",
    "".join(ds["variant_allele"][:][np.arange(len(BA_1_haplotype)), BA_1_haplotype][11280: 11300])
)
# Set all deletions as missing, apart from the first at this site
deletion_idx = np.where(ds["variant_allele"][0, :] == "-")[0][0]
BA_1_haplotype[BA_1_haplotype == deletion_idx] = tskit.MISSING_DATA
BA_1_haplotype[11287:(11287+nmut)] = deletion_idx
print(
    "Missing ",
    "".join(ds["variant_allele"][:][np.arange(len(BA_1_haplotype)), BA_1_haplotype][11280: 11300])
)

## BA.2
print(f"\nBA.2: {BA_2}")
# Keep the deletion in BA.2 in the Alpha position: we will assign the BA.1 deletions to start at
# 11288 in the pre-BA.2 tree sequence
print(
    "Was     ", 
    "".join(ds["variant_allele"][:][np.arange(len(BA_2_haplotype)), BA_2_haplotype][11280: 11300])
)
print(
    "Now     ",
    "".join(ds["variant_allele"][:][np.arange(len(BA_2_haplotype)), BA_2_haplotype][11280: 11300])
)
# Set all deletions as missing, apart from the first at this site
deletion_idx = np.where(ds["variant_allele"][0, :] == "-")[0][0]
BA_2_haplotype[BA_2_haplotype == deletion_idx] = tskit.MISSING_DATA
BA_2_haplotype[11287:(11287+nmut)] = deletion_idx
print(
    "Missing ",
    "".join(ds["variant_allele"][:][np.arange(len(BA_2_haplotype)), BA_2_haplotype][11280: 11300])
)

BA.1: ERR7602255
Was      TA---------GTTTTAAGC
Now      TAGTTTT---------AAGC
Missing  TAGTTTT---......AAGC

BA.2: SRR17712694
Was      TAGTTTG---------AAGC
Now      TAGTTTG---------AAGC
Missing  TAGTTTG---......AAGC


In [5]:
import pandas as pd
path = "synthetic_align.zarr"
sc2ts.Dataset.new(path)
sc2ts.Dataset.append_alignments(path, {BA_1: BA_1_haplotype})
sc2ts.Dataset.append_alignments(path, {BA_2: BA_2_haplotype})
sc2ts.Dataset.add_metadata(path, pd.DataFrame(
    {'date': [ts.node(BA_1_node).metadata["date"], ts.node(BA_2_node).metadata["date"]]},
    index=[BA_1, BA_2],
))
synth_ds = sc2ts.Dataset(path)
print("Loaded", synth_ds)

Loaded Dataset at synthetic_align.zarr with 2 samples and 1 metadata fields


Add the deletion into the tree sequence at the Alpha position 11288

In [6]:
# Check the nodes look the same in the pre_BA_1 ts
assert ts.node(Alpha_node).metadata["sc2ts"] == ts_pre_BA_1.node(Alpha_node).metadata["sc2ts"]
tables = ts_pre_BA_1.dump_tables()
for pos in range(11288, 11288+nmut):
    tables.mutations.add_row(node=Alpha_node, time=ts_pre_BA_1.node(Alpha_node).time, site=ts.site(position=pos).id, derived_state="-")
tables.sort()
ts_pre_BA_1_del = tables.tree_sequence()

print("Check that there is a GTTTG before the newly created deletion")
for align in ts_pre_BA_1_del.simplify([1, Alpha_node]).alignments(left=11280, right=11300):
    print(align)

Check that there is a GTTTG before the newly created deletion
CTAGTTTGTCTGGTTTTAAG
CTAGTTTG---GGTTTTAAG


In [7]:
# Check the nodes look the same in the pre_BA_2 ts
assert ts.node(BA_1_node).metadata["sc2ts"] == ts_pre_BA_2.node(BA_1_node).metadata["sc2ts"]
tables = ts_pre_BA_2.dump_tables()
for pos in range(11288, 11288+nmut):
    tables.mutations.add_row(node=Alpha_node, time=ts_pre_BA_2.node(Alpha_node).time, site=ts.site(position=pos).id, derived_state="-")
    tables.mutations.add_row(node=BA_1_node, time=ts_pre_BA_2.node(BA_1_node).time, site=ts.site(position=pos).id, derived_state="-")
tables.sort()
ts_pre_BA_2_del = tables.tree_sequence()

print("Check that there is a GTTTG before the newly created deletion")
for align in ts_pre_BA_2_del.simplify([1, BA_1_node]).alignments(left=11280, right=11300):
    print(align)

Check that there is a GTTTG before the newly created deletion
CTAGTTTGTCTGGTTTTAAG
CTAGTTTG---GGTTTTAAG


# Run one round of sc2ts

(first define a `match` function to use: code taken from https://github.com/jeromekelleher/sc2ts/blob/89421fb4cc6bd8d9cf89a6cc8bdf70ea8cc95c1a/sc2ts/cli.py#L835)

In [8]:
import json
import dataclasses


@dataclasses.dataclass(frozen=True)
class HmmRun:
    strain: str
    num_mismatches: int
    direction: str
    match: sc2ts.HmmMatch

    def asdict(self):
        d = dataclasses.asdict(self)
        d["match"] = dataclasses.asdict(self.match)
        d["match"]["cost"] = d["match"]["cost"].item()
        d["match"]["likelihood"] = d["match"]["likelihood"].item()
        return d

    def asjson(self):
        return json.dumps(self.asdict())

def hmm_match(ds, strains, ts, lineage_imputed_ts=None):
    args = json.loads(ts.provenance(-1).record)['parameters']['args']
    mismatch_threshold= 100  # Not sure what this should be
    samples = sc2ts.preprocess(
        strains,
        dataset=synth_ds,
        keep_sites=ts.sites_position.astype(int),
    )
    for sample in samples:
        if sample.haplotype is None:
            raise ValueError(f"No alignment stored for {sample.strain}")
    
    run = {}
    print(
        "Deletion modelled as",
        "missing data" if nmut==0 else f"{nmut} SNP{'s' if nmut > 0 else ''}"
    )
    for direction in ("forward", "reverse"):
        sc2ts.match_tsinfer(
            samples=samples,
            ts=ts,
            num_mismatches=int(args[args.index("--num-mismatches") + 1]),
            deletions_as_missing=False,
            mismatch_threshold=mismatch_threshold,
            mirror_coordinates=direction == "reverse",
        )
        for sample in samples:
            run[direction] = HmmRun(
                strain=sample.strain,
                num_mismatches=int(args[args.index("--num-mismatches") + 1]),
                direction=direction,
                match=sample.hmm_match,
            )
        paths = []
        for p in run[direction].match.path:
            paths.append(f"[left={p.left}, right={p.right}, parent=node {p.parent}")
            if lineage_imputed_ts is not None:
                parent = lineage_imputed_ts.node(p.parent)
                paths[-1] += f"({parent.metadata.get('Imputed_Viridian_pangolin')})]"
        muts = [
            str(m)
            for m in run[direction].match.mutations if m.derived_state=="-"
        ]
        print(
            f"Matching {direction}: {", ".join(paths)}",
            f"Del muts:",
            ", ".join(muts)  or "None",
            f"(likelihood={run[direction].match.likelihood}, cost={run[direction].match.cost})",
        )
    return run

## Alpha vs BA.1

Match the BA.1 synth sample (with the deletion) into the pre_BA.1 ARG, which has the Alpha deletion

In [9]:
# Should check that any deletion mutations start at the Alpha location, 11288
BA1_into_Alpha = hmm_match(synth_ds, [BA_1], ts_pre_BA_1_del, ts)

Deletion modelled as 3 SNPs
Matching forward: [left=0, right=29904, parent=node 4471(B.1.1)] Del muts: T11288-, C11289-, T11290- (likelihood=3.587324068671541e-90, cost=47)
Matching reverse: [left=0, right=29904, parent=node 18560(B.1.1)] Del muts: T11288-, C11289-, T11290- (likelihood=3.587324068671541e-90, cost=47)


### Discussion
Adjust the value of `nmut` (e.g. set `nmut=5` in the early cells) to show that it requires the deletion to be modelled by at least 5 SNPs before we deduce a single deletion origin (in that case, it is then explained by recombining the deletion region out of an Alpha / B.1.1.7 strain)

## BA.1 vs BA.2

Match the BA.2 sample (with the deletion) into the pre_BA.2 ARG, in which the BA.1 deletion has been moved into the
Alpha position.

In [10]:
BA2_into_BA_1 = hmm_match(synth_ds, [BA_2], ts_pre_BA_2_del, ts)

Deletion modelled as 3 SNPs
Matching forward: [left=0, right=22674, parent=node 67513(B.1)], [left=22674, right=29904, parent=node 338914(BA.1.1)] Del muts: T11288-, C11289-, T11290- (likelihood=6.502475810729579e-73, cost=38)
Matching reverse: [left=0, right=9345, parent=node 67513(B.1)], [left=9345, right=29904, parent=node 338914(BA.1.1)] Del muts: None (likelihood=6.502475810729579e-73, cost=38)


### Discussion
It appears as if with `nmuts=3`, the forward run picks the deletion region from the LH parent (not BA.1.1), and hence requires 3 deletions, whereas the reverse run takes this region from the RH parent, which is BA.1.1 and hence already has the deletion.